# 🧠 Brain Trust - Iris Tools Tester

Test the Librarian (Iris) Google Drive tools from anywhere.

## Setup (One Time)
1. Upload your `credentials.json` to Google Drive (in a safe folder)
2. Update the `CREDENTIALS_PATH` below to point to it
3. Run all cells

## Usage
- Run the setup cells first
- Then use any of the test cells to interact with your Drive

In [ ]:
#@title 📦 Install Dependencies
!pip install -q google-api-python-client google-auth google-auth-oauthlib

In [ ]:
#@title 🔗 Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('✅ Google Drive connected!')

In [ ]:
#@title ⚙️ Configuration

# UPDATE THIS PATH to where you stored credentials.json in your Drive
# Example: /content/drive/MyDrive/BrainTrust/credentials.json
CREDENTIALS_PATH = "/content/drive/MyDrive/credentials.json"  #@param {type:"string"}

# Folder IDs for the editorial pipeline
FOLDER_IDS = {
    'life_with_ai': '1duf6BRY-tqyWzP3gH1clfaM5B-Qqh0r5',
    'inbox': '1KzQODiGWI0DTsV6dEJHOGvniKx7nR8Ss',
    'in_development': '1fKYixOC9aDcm-XHAIHhfGj3rKlRg5b-i',
    'ready_for_review': '1Zy2ocnE4EOwpeuvu8RFrNS_WophR4ebB',
    'published': '1buuMweoHEdo17_YdR9qgXw42dpNhgyR3',
    'voice_library': '1zzD3HBIrjOAQ-L5a1WDsewroKaOzpie_',
    'agent_prompts': '1u81lD6187CGxztciR_clM4rH-9TpM1TT',
    'reference_docs': '1SKO7KfS3UUuwTiHQ1WYS0eBfVryR1Wxr',
    'workflows': '1ZGHfjdxIJPxOe_W_8iNskV9U5UkT0uAc',
    'images': '1y7yCgVcFsgWSTk4ZreuMBcgfJ91il6MO',
}

print('✅ Configuration loaded!')

In [ ]:
#@title 🔐 Initialize Google APIs
from google.oauth2 import service_account
from googleapiclient.discovery import build
import os

SCOPES = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/documents'
]

# Check credentials file exists
if not os.path.exists(CREDENTIALS_PATH):
    print(f'❌ ERROR: credentials.json not found at {CREDENTIALS_PATH}')
    print('   Please upload it to your Google Drive and update CREDENTIALS_PATH')
else:
    creds = service_account.Credentials.from_service_account_file(
        CREDENTIALS_PATH, scopes=SCOPES
    )
    drive_service = build('drive', 'v3', credentials=creds)
    docs_service = build('docs', 'v1', credentials=creds)
    print(f'✅ Authenticated as: {creds.service_account_email}')

---
## 🔧 Iris Tools

Run any of these cells to test Iris's capabilities.

In [ ]:
#@title 📊 Pipeline Status
from datetime import datetime, timedelta

pipeline_folders = ['inbox', 'in_development', 'ready_for_review', 'published']
counts = {}
stale_files = []
stale_threshold = datetime.utcnow() - timedelta(days=7)

for folder_name in pipeline_folders:
    folder_id = FOLDER_IDS.get(folder_name)
    if not folder_id:
        continue

    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed = false",
        pageSize=100,
        fields="files(id, name, modifiedTime)"
    ).execute()

    files = results.get('files', [])
    counts[folder_name] = len(files)

    if folder_name == 'in_development':
        for f in files:
            if f.get('modifiedTime'):
                mod_time = datetime.fromisoformat(f['modifiedTime'].replace('Z', '+00:00'))
                if mod_time.replace(tzinfo=None) < stale_threshold:
                    days_old = (datetime.utcnow() - mod_time.replace(tzinfo=None)).days
                    stale_files.append((f['name'], days_old))

print("📊 PIPELINE STATUS")
print("═" * 40)
print(f"\n📥 Inbox:            {counts.get('inbox', 0)} items")
print(f"🔧 In Development:   {counts.get('in_development', 0)} items")
print(f"👁️ Ready for Review: {counts.get('ready_for_review', 0)} items")
print(f"✅ Published:        {counts.get('published', 0)} items")
print(f"\n   Total: {sum(counts.values())} files in pipeline")

if stale_files:
    print(f"\n⚠️ STALE FILES (>7 days in Development):")
    for name, days in stale_files:
        print(f"   - {name} ({days} days)")

if counts.get('inbox', 0) > 0:
    print(f"\n📌 Note: {counts['inbox']} items in Inbox awaiting triage")

In [ ]:
#@title 🔍 Find Files
search_query = "Arun"  #@param {type:"string"}

results = drive_service.files().list(
    q=f"name contains '{search_query}' and trashed = false",
    pageSize=20,
    fields="files(id, name, mimeType, modifiedTime)"
).execute()

items = results.get('files', [])

if not items:
    print(f"No files found matching '{search_query}'")
else:
    print(f"Found {len(items)} file(s) matching '{search_query}':\n")
    for item in items:
        icon = "📄" if 'document' in item.get('mimeType', '') else "📁" if 'folder' in item.get('mimeType', '') else "📎"
        mod = item.get('modifiedTime', '')[:10] if item.get('modifiedTime') else ''
        print(f"{icon} {item['name']}")
        print(f"   ID: {item['id']}")
        print(f"   Modified: {mod}\n")

In [ ]:
#@title 📂 List Folder Contents
folder_name = "inbox"  #@param ["inbox", "in_development", "ready_for_review", "published", "reference_docs", "voice_library"]

folder_id = FOLDER_IDS.get(folder_name)
if not folder_id:
    print(f"Unknown folder: {folder_name}")
else:
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and trashed = false",
        pageSize=50,
        fields="files(id, name, mimeType)"
    ).execute()

    items = results.get('files', [])
    folders = [i for i in items if 'folder' in i.get('mimeType', '')]
    files = [i for i in items if 'folder' not in i.get('mimeType', '')]

    print(f"📂 Contents of '{folder_name}':")
    print(f"   {len(folders)} folders, {len(files)} files\n")

    if folders:
        print("📁 FOLDERS:")
        for f in folders:
            print(f"   - {f['name']}")
        print()

    if files:
        print("📄 FILES:")
        for f in files:
            print(f"   - {f['name']}")
            print(f"     ID: {f['id']}")

In [ ]:
#@title 📖 Read Document
document_id = ""  #@param {type:"string"}

if not document_id:
    print("Please enter a document ID (get it from 'Find Files' or 'List Folder')")
else:
    try:
        doc = docs_service.documents().get(documentId=document_id).execute()
        title = doc.get('title', 'Untitled')

        # Extract text
        text = ""
        for content in doc.get('body', {}).get('content', []):
            if 'paragraph' in content:
                for elem in content['paragraph'].get('elements', []):
                    if 'textRun' in elem:
                        text += elem['textRun'].get('content', '')

        print(f"📖 {title}")
        print("═" * 40)
        print(text[:2000])  # First 2000 chars
        if len(text) > 2000:
            print(f"\n... ({len(text) - 2000} more characters)")
    except Exception as e:
        print(f"Error reading document: {e}")

In [ ]:
#@title ✨ Create New Document (via Template Copy)
document_title = "Arun_Story_Draft_1"  #@param {type:"string"}
target_folder = "inbox"  #@param ["inbox", "in_development", "ready_for_review"]
content = "# New Document\n\nCreated by Iris from Colab."  #@param {type:"string"}

# Find template
print("1. Looking for template...")
results = drive_service.files().list(
    q="name contains '_BrainTrust_Template' and mimeType = 'application/vnd.google-apps.document' and trashed = false",
    pageSize=5,
    fields="files(id, name)"
).execute()

templates = results.get('files', [])
if not templates:
    print("❌ No template found! Please create a document named '_BrainTrust_Template' in your Drive.")
else:
    template_id = templates[0]['id']
    print(f"   Found: {templates[0]['name']}")

    # Copy template
    print(f"\n2. Creating '{document_title}' in {target_folder}...")
    folder_id = FOLDER_IDS.get(target_folder)

    new_doc = drive_service.files().copy(
        fileId=template_id,
        body={'name': document_title, 'parents': [folder_id]},
        fields='id, name, webViewLink'
    ).execute()

    doc_id = new_doc['id']
    print(f"   Created: {new_doc['name']}")

    # Add content
    print("\n3. Adding content...")
    doc = docs_service.documents().get(documentId=doc_id).execute()
    doc_content = doc.get('body', {}).get('content', [])

    end_index = 1
    for element in doc_content:
        if 'endIndex' in element:
            end_index = max(end_index, element['endIndex'])

    requests = []
    if end_index > 2:
        requests.append({'deleteContentRange': {'range': {'startIndex': 1, 'endIndex': end_index - 1}}})
    requests.append({'insertText': {'location': {'index': 1}, 'text': content}})

    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

    print(f"\n✅ SUCCESS!")
    print(f"   Document: {document_title}")
    print(f"   Folder: {target_folder}")
    print(f"   View: https://docs.google.com/document/d/{doc_id}/edit")

In [ ]:
#@title 📦 Move File
file_id = ""  #@param {type:"string"}
destination = "in_development"  #@param ["inbox", "in_development", "ready_for_review", "published"]

if not file_id:
    print("Please enter a file ID to move")
else:
    # Get current file info
    file_info = drive_service.files().get(
        fileId=file_id,
        fields='id, name, parents'
    ).execute()

    file_name = file_info.get('name', 'Unknown')
    current_parents = file_info.get('parents', [])
    dest_folder_id = FOLDER_IDS.get(destination)

    # Move file
    old_parents = ','.join(current_parents) if current_parents else None

    drive_service.files().update(
        fileId=file_id,
        addParents=dest_folder_id,
        removeParents=old_parents,
        fields='id, parents'
    ).execute()

    print(f"✅ Moved '{file_name}' to '{destination}'")

In [ ]:
#@title 🏷️ Rename File
file_id = ""  #@param {type:"string"}
new_name = ""  #@param {type:"string"}

if not file_id or not new_name:
    print("Please enter both file ID and new name")
else:
    file_info = drive_service.files().get(fileId=file_id, fields='name').execute()
    old_name = file_info.get('name', 'Unknown')

    drive_service.files().update(
        fileId=file_id,
        body={'name': new_name}
    ).execute()

    print(f"✅ Renamed '{old_name}' → '{new_name}'")

In [ ]:
#@title 📋 Recent Changes
days_back = 7  #@param {type:"integer"}

from datetime import datetime, timedelta

cutoff = datetime.utcnow() - timedelta(days=days_back)
cutoff_str = cutoff.strftime('%Y-%m-%dT%H:%M:%S')

results = drive_service.files().list(
    q=f"modifiedTime > '{cutoff_str}' and trashed = false",
    pageSize=50,
    orderBy='modifiedTime desc',
    fields="files(id, name, mimeType, modifiedTime, parents)"
).execute()

items = results.get('files', [])

print(f"📋 FILES MODIFIED IN LAST {days_back} DAYS")
print("═" * 40)

if not items:
    print(f"No files modified in the last {days_back} days.")
else:
    print(f"Found {len(items)} file(s):\n")
    for item in items:
        icon = "📄" if 'document' in item.get('mimeType', '') else "📁" if 'folder' in item.get('mimeType', '') else "📎"
        mod = item.get('modifiedTime', '')[:10] if item.get('modifiedTime') else ''

        # Get folder name
        folder_name = "Unknown"
        if item.get('parents'):
            parent_id = item['parents'][0]
            for name, fid in FOLDER_IDS.items():
                if fid == parent_id:
                    folder_name = name
                    break

        print(f"{icon} {item['name']}")
        print(f"   Modified: {mod} | Folder: {folder_name}")